# Imports

In [1]:
from pathlib import Path
from functools import reduce

import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.formula.api import ols

from more_itertools import powerset

from constants import DataSplit, Model, METRICS, DATASET_SYMBOLS

# Parameters

In [2]:
data_dir = r'./out'
concat_results = True
alpha = 0.05

# Data Loading

In [3]:
path = Path(data_dir)

pattern = rf'{"**/*" if concat_results else ""}results.csv'

data = pd.concat([pd.read_csv(filename, index_col=0) for filename in path.glob(pattern)], ignore_index=True)
display(data)

,DecisionTree,RandomForest,SVM,KNN,LogisticRegression,RandomBaseline,ConstantBaseline,PreviousBaseline,ConsensusBaseline,forex,bond,index_futures,commodities_futures,SPY,Random,test,accuracy,weighted f1,macro f1,roc-auc
0,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0.833458,0.832359,0.830981,0.933731
1,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,0.514970,0.508860,0.498796,0.491255
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,0.784738,0.777865,0.774654,0.888079
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,0.547904,0.420897,0.385600,0.499613
4,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,0.700646,0.700834,0.697850,0.767178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9175,False,False,False,False,False,False,False,False,True,True,False,True,True,True,False,True,0.495475,0.501318,0.475140,NaN
9176,False,False,False,False,False,False,False,False,True,False,True,True,True,True,False,False,0.506523,0.510346,0.493364,NaN
9177,False,False,False,False,False,False,False,False,True,False,True,True,True,True,False,True,0.503145,0.511602,0.474699,NaN
9178,False,False,False,False,False,False,False,False,True,True,True,True,True,True,False,False,0.506523,0.510346,0.493364,NaN


# Asset presence analysis (model-averaged, baseline models and training data excluded)

In [4]:
df = data.loc[data[DataSplit.TEST] & ~data[Model.RANDOM_BASELINE] & ~data[Model.CONSTANT_BASELINE] & ~data[Model.PREVIOUS_BASELINE] & ~data[Model.CONSENSUS_BASELINE]]
df = df.drop(columns=[DataSplit.TEST])

pivot = [
    df.loc[df['Random']]
    .pivot_table(values=[metric for metric in METRICS.keys()],
                 index='Random',
                 aggfunc=[np.mean, np.std]),
    df[~df[[asset_type for asset_type in DATASET_SYMBOLS.keys()] + ['Random']].any(axis=1)]
    .pivot_table(values=[metric for metric in METRICS.keys()],
                 index='SPY',
                 aggfunc=[np.mean, np.std])
]

for asset_type in DATASET_SYMBOLS.keys():
    pivot.append(df.loc[~df['Random']].pivot_table(values=[metric for metric in METRICS.keys()], index=asset_type, aggfunc=[np.mean, np.std]))

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['asset type','presence']).rename(index={'SPY':'SPY-Only', 'Random':'Random Data'})

display(pivot)

mean                             std  \
                              accuracy  macro f1 weighted f1  accuracy   
asset type          presence                                             
Random Data         True      0.512109  0.435936    0.446795  0.036709   
SPY-Only            True      0.530632  0.467813    0.485363  0.022562   
forex               False     0.538441  0.484100    0.503451  0.027006   
                    True      0.531317  0.476615    0.496262  0.029520   
bond                False     0.532666  0.473000    0.493073  0.026694   
                    True      0.537092  0.487714    0.506639  0.030062   
index_futures       False     0.531650  0.475185    0.494990  0.027238   
                    True      0.538109  0.485530    0.504723  0.029383   
commodities_futures False     0.530491  0.476512    0.493806  0.024761   
                    True      0.539268  0.484203    0.505906  0.031217   

                                                    
                              macro f1 weighted f1  
asset type          presence                        
Random Data         True      0.059279    0.058084  
SPY-Only            True      0.046459    0.035177  
forex               False     0.050217    0.039871  
                    True      0.045346    0.035467  
bond                False     0.051098    0.039499  
                    True      0.043435    0.034946  
index_futures       False     0.048321    0.037101  
                    True      0.047091    0.038074  
commodities_futures False     0.047428    0.037657  
                    True      0.048239    0.037178

# Asset combinations (model-averaged, baselines and training data excluded)

In [5]:
df = data.loc[data[DataSplit.TEST] & ~data['Random'] & ~data[Model.RANDOM_BASELINE] & ~data[Model.CONSTANT_BASELINE] & ~data[Model.PREVIOUS_BASELINE] & ~data[Model.CONSENSUS_BASELINE]]
df = df.drop(columns=[DataSplit.TEST, 'Random'])

df['forex'] = df['forex'].map({True:'F',False:''})
df['bond'] = df['bond'].map({True:'B',False:''})
df['index_futures'] = df['index_futures'].map({True:'I',False:''})
df['commodities_futures'] = df['commodities_futures'].map({True:'C',False:''})

df['asset combination'] = df['forex'] + df['bond'] + df['index_futures'] + df['commodities_futures']

pivot = df.pivot_table(values=[metric for metric in METRICS.keys()],
                       index='asset combination',
                       aggfunc=[np.mean, np.std])

pivot = pivot.reindex([reduce(lambda x, y: x+y, c, '') for c in powerset('FBIC')])
pivot.rename(index={'':'SPY-Only'}, inplace=True)


display(pivot)

mean                             std            \
                   accuracy  macro f1 weighted f1  accuracy  macro f1   
asset combination                                                       
SPY-Only           0.530632  0.467813    0.485363  0.022562  0.046459   
F                  0.518958  0.463498    0.479864  0.021717  0.054509   
B                  0.535652  0.487162    0.504061  0.022217  0.045115   
I                  0.532923  0.478709    0.495668  0.020642  0.044504   
C                  0.530920  0.462975    0.488646  0.029101  0.052304   
FB                 0.528792  0.480746    0.497976  0.025686  0.041274   
FI                 0.540850  0.486778    0.503631  0.025888  0.050557   
FC                 0.535683  0.460027    0.487496  0.024494  0.052067   
BI                 0.532202  0.474412    0.493262  0.023183  0.054507   
BC                 0.548724  0.504010    0.522066  0.028653  0.040939   
IC                 0.540462  0.486902    0.506671  0.026982  0.054628   
FBI                0.523919  0.472981    0.490622  0.029013  0.034900   
FBC                0.523838  0.475251    0.494445  0.031762  0.034816   
FIC                0.530902  0.477302    0.497246  0.033837  0.045695   
BIC                0.556017  0.510818    0.531867  0.029819  0.042140   
FBIC               0.547595  0.496336    0.518813  0.031079  0.033287   

                               
                  weighted f1  
asset combination              
SPY-Only             0.035177  
F                    0.042623  
B                    0.034273  
I                    0.035329  
C                    0.038637  
FB                   0.032195  
FI                   0.042385  
FC                   0.037744  
BI                   0.043632  
BC                   0.031528  
IC                   0.041493  
FBI                  0.026366  
FBC                  0.025472  
FIC                  0.034754  
BIC                  0.033646  
FBIC                 0.023287

# Model Performance (dataset averaged, random data excluded)
## Out-sample

In [6]:
df = data.loc[~data['Random']]

pivot = []

for model in Model:
    pivot.append(df.loc[df[DataSplit.TEST]].pivot_table(values=[metric for metric in METRICS.keys()], index=model, aggfunc=[np.mean, np.std]))

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['model','used'])
pivot = pivot.loc[pivot.index.get_level_values(1)].droplevel(1)

display(pivot)

mean                             std            \
                    accuracy  macro f1 weighted f1  accuracy  macro f1   
model                                                                    
DecisionTree        0.509493  0.498306    0.507191  0.023638  0.024257   
RandomForest        0.536710  0.489942    0.510284  0.019680  0.021839   
SVM                 0.569525  0.400804    0.442695  0.011573  0.029455   
KNN                 0.522114  0.501280    0.513969  0.022972  0.023785   
LogisticRegression  0.536555  0.511455    0.525142  0.021087  0.029430   
RandomBaseline      0.516363  0.512286    0.518188  0.025874  0.025665   
ConstantBaseline    0.566104  0.361456    0.409296  0.008064  0.003286   
PreviousBaseline    0.535565  0.527326    0.535495  0.020931  0.020599   
ConsensusBaseline   0.495972  0.474116    0.503530  0.010378  0.007764   

                                
                   weighted f1  
model                           
DecisionTree          0.023137  
RandomForest          0.019988  
SVM                   0.028081  
KNN                   0.022363  
LogisticRegression    0.026304  
RandomBaseline        0.025957  
ConstantBaseline      0.009557  
PreviousBaseline      0.020902  
ConsensusBaseline     0.011041

## In-sample

In [7]:
df = data.loc[~data['Random']]

pivot = []

for model in Model:
    pivot.append(df.loc[~df[DataSplit.TEST]].pivot_table(values=[metric for metric in METRICS.keys()], index=model, aggfunc=[np.mean, np.std]))

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['model','used'])
pivot = pivot.loc[pivot.index.get_level_values(1)].droplevel(1)

display(pivot)

mean                             std            \
                    accuracy  macro f1 weighted f1  accuracy  macro f1   
model                                                                    
DecisionTree        0.826879  0.823471    0.825665  0.098656  0.102646   
RandomForest        0.999061  0.999048    0.999058  0.005230  0.005303   
SVM                 0.571408  0.409397    0.441027  0.023597  0.046400   
KNN                 0.711307  0.704566    0.709061  0.112728  0.114865   
LogisticRegression  0.638873  0.622681    0.629963  0.057780  0.067984   
RandomBaseline      0.501323  0.499650    0.502654  0.008184  0.008220   
ConstantBaseline    0.551458  0.355430    0.392056  0.007494  0.003138   
PreviousBaseline    0.477660  0.471936    0.477672  0.008786  0.009736   
ConsensusBaseline   0.507286  0.496815    0.510188  0.007241  0.006882   

                                
                   weighted f1  
model                           
DecisionTree          0.100892  
RandomForest          0.005253  
SVM                   0.045223  
KNN                   0.113276  
LogisticRegression    0.066294  
RandomBaseline        0.008111  
ConstantBaseline      0.008713  
PreviousBaseline      0.008778  
ConsensusBaseline     0.007451

# Statistical analysis of effects

In [19]:
all_df = data.loc[data[DataSplit.TEST] & ~data['Random']]
all_df = all_df.drop(columns=[DataSplit.TEST, 'SPY', 'Random'])
all_df = all_df.replace({True: 1, False: -1}) # required to get coefficients (for identifying direction)b

for model in Model:
    df = all_df.loc[data[model]].drop(columns=[m.value for m in Model])

    for metric in METRICS.keys():
        relation = f'Q("{metric}") ~ forex * bond * index_futures * commodities_futures'
        glm = ols(relation, data=df).fit()
        aov = sm.stats.anova_lm(glm, typ=1)

        coefs = glm.params.rename(index={'Intercept':'Residual/Intercept'})
        coefs.name = 'coefs'
        aov = aov.rename(index={'Residual':'Residual/Intercept'})

        print(f'{model}: {metric} (full model)')
        display(aov.join(coefs))

        while (aov['PR(>F)'] > alpha).any():
            rem_effect = aov['F'].idxmin().split(':')
            # print(f'removing effect {":".join(rem_effect)} (p={aov["PR(>F)"].max():.3f})')
            for effect in aov.index:
                if all(main_effect in effect.split(':') for main_effect in rem_effect):
                    relation += f' - {effect}'


            glm = ols(relation, data=df).fit()
            aov = sm.stats.anova_lm(glm, typ=1)

        coefs = glm.params.rename(index={'Intercept':'Residual/Intercept'})
        coefs.name = 'coefs'
        aov = aov.rename(index={'Residual':'Residual/Intercept'})

        print(f'\n{model}: {metric} (reduced model)')
        display(aov.join(coefs))

DecisionTree: accuracy (full model)


,df,sum_sq,mean_sq,F,PR(>F),coefs
forex,1.0,0.000017,0.000017,0.030347,0.861782,-0.000187
bond,1.0,0.000309,0.000309,0.560085,0.454605,0.000802
forex:bond,1.0,0.000649,0.000649,1.175266,0.278885,0.001162
index_futures,1.0,0.001528,0.001528,2.768358,0.096820,0.001784
forex:index_futures,1.0,0.000419,0.000419,0.758553,0.384232,0.000934
bond:index_futures,1.0,0.000107,0.000107,0.193616,0.660129,0.000472
forex:bond:index_futures,1.0,0.002298,0.002298,4.164426,0.041847,0.002188
commodities_futures,1.0,0.000021,0.000021,0.038887,0.843759,-0.000211
forex:commodities_futures,1.0,0.001446,0.001446,2.620068,0.106199,0.001736
bond:commodities_futures,1.0,0.002184,0.002184,3.957245,0.047257,0.002133



DecisionTree: accuracy (reduced model)


,df,sum_sq,mean_sq,F,PR(>F),coefs
Residual/Intercept,479.0,0.267633,0.000559,NaN,NaN,0.509493


DecisionTree: weighted f1 (full model)


...

,df,sum_sq,mean_sq,F,PR(>F),coefs
forex,1.0,1.356491e-06,1.356491e-06,7.958533e+23,0.0,0.000053
bond,1.0,6.061858e-04,6.061858e-04,3.556493e+26,0.0,-0.001124
forex:bond,1.0,1.356491e-06,1.356491e-06,7.958533e+23,0.0,-0.000053
index_futures,1.0,1.648756e-03,1.648756e-03,9.673257e+26,0.0,-0.001853
forex:index_futures,1.0,1.356491e-06,1.356491e-06,7.958533e+23,0.0,-0.000053
bond:index_futures,1.0,2.704959e-03,2.704959e-03,1.587000e+27,0.0,0.002374
forex:bond:index_futures,1.0,1.356491e-06,1.356491e-06,7.958533e+23,0.0,0.000053
commodities_futures,1.0,3.797546e-02,3.797546e-02,2.228021e+28,0.0,0.008895
forex:commodities_futures,1.0,1.356491e-06,1.356491e-06,7.958533e+23,0.0,-0.000053
bond:commodities_futures,1.0,1.291592e-07,1.291592e-07,7.577775e+22,0.0,-0.000016


ConsensusBaseline: macro f1 (full model)


,df,sum_sq,mean_sq,F,PR(>F),coefs
forex,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,0.000064
bond,1.0,4.921567e-03,4.921567e-03,3.839358e+27,0.0,-0.003202
forex:bond,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,-0.000064
index_futures,1.0,5.814776e-04,5.814776e-04,4.536158e+26,0.0,-0.001101
forex:index_futures,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,-0.000064
bond:index_futures,1.0,2.957296e-05,2.957296e-05,2.307013e+25,0.0,0.000248
forex:bond:index_futures,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,0.000064
commodities_futures,1.0,1.384017e-02,1.384017e-02,1.079684e+28,0.0,0.005370
forex:commodities_futures,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,-0.000064
bond:commodities_futures,1.0,1.309196e-06,1.309196e-06,1.021315e+24,0.0,-0.000052



ConsensusBaseline: macro f1 (reduced model)


,df,sum_sq,mean_sq,F,PR(>F),coefs
forex,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,0.000064
bond,1.0,4.921567e-03,4.921567e-03,3.839358e+27,0.0,-0.003202
forex:bond,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,-0.000064
index_futures,1.0,5.814776e-04,5.814776e-04,4.536158e+26,0.0,-0.001101
forex:index_futures,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,-0.000064
bond:index_futures,1.0,2.957296e-05,2.957296e-05,2.307013e+25,0.0,0.000248
forex:bond:index_futures,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,0.000064
commodities_futures,1.0,1.384017e-02,1.384017e-02,1.079684e+28,0.0,0.005370
forex:commodities_futures,1.0,1.994834e-06,1.994834e-06,1.556187e+24,0.0,-0.000064
bond:commodities_futures,1.0,1.309196e-06,1.309196e-06,1.021315e+24,0.0,-0.000052
